In this notebook, I try to compute the precipitation estimated by the neural network cumulus convective scheme. I compare the precipitations estimated from the temperature source and moisture sink, and also look at their exponential dependence compared to PW and column relative humidity.

In [ ]:
import dask.array as da
import holoviews as hv
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from holoviews.operation.datashader import datashade, dynspread
from matplotlib.colors import LogNorm
from sklearn.externals import joblib

import torch
from lib.evaluation.single_column import predict
from lib.models.torch.preprocess import stacked_data, unstacked_data
from lib.thermo import (column_rh, liquid_water_temperature, mass_integrate,
                        total_water)
from metpy.calc import mixed_layer, relative_humidity_from_mixing_ratio
from metpy.units import units as u

%matplotlib inline
hv.extension('matplotlib')

# Computing precipitation

In [ ]:
# load torch stepper
stepper = torch.load("../data/ml/ngaqua/multistep_objective.torch")

# load coarse-grained data
data_3d = xr.open_mfdataset("../data/raw/ngaqua/coarse/3d/*.nc",
                            preprocess=lambda x: x.drop('p')).isel(y=slice(24,40)).compute()
p = xr.open_dataset("../data/raw/ngaqua/stat.nc").p
p.attrs['units'] = 'hPa'

# compute the inputs for the torch stepper
inputs = xr.Dataset(dict(sl=liquid_water_temperature(data_3d.TABS, data_3d.QN, data_3d.QP), 
                     qt=total_water(data_3d.QV, data_3d.QN)))
X = stacked_data(inputs.transpose('time', 'y', 'x', 'z'))

# make the prediction of heatings
Q = predict(stepper.rhs, X)
Q = unstacked_data(Q)
Q = xr.Dataset({
    'Q1': (['time', 'y', 'x', 'z'], Q['sl']*1005/2.5e6),
    'Q2': (['time', 'y', 'x', 'z'], Q['qt']/1000)},
    coords=data_3d.coords)

# compute diagnostics
# PW, column RH, Q1 and Q2 int
pw = mass_integrate(p, inputs.qt/1000)
q_int = mass_integrate(p, Q)
rh_col = column_rh(data_3d.QV, data_3d.TABS, p)

ds = xr.Dataset({'PW': pw, 'PRECQ': -q_int.Q2, 'PRECT': q_int.Q1, 'RH': rh_col})
ds.time.attrs['units'] = 'days since 2000-01-01'
ds = xr.decode_cf(ds)

In [ ]:
%%output size=300
%%opts Image[colorbar=True](cmap='inferno')

mu = ds.mean('time')

hv.HoloMap({'PRECQ': hv.Dataset(mu.PRECQ).to(hv.Image),
            'PRECT': hv.Dataset(mu.PRECT).to(hv.Image)})


In [ ]:
%%output size=300
%%opts Curve[aspect=5](linewidth=.5)
hv.Dataset(q_int.mean(['x', 'y'])).to.curve("time")

In [ ]:
mu = ds.mean(['x' ,'time'])[['PRECQ', 'PRECT']]

mu.to_dataframe().plot()

The difference between these curves is probably just $LHF-(SHF + QRAD)$

# Relationship between precip and PW

From Bretherton, Peters, and Back (2004) precipitation should show an approximately exponential relationship with RH.

In [ ]:
def plot_scatter(x, y):
    plt.hexbin(x, y,  norm=LogNorm())
    plt.ylim([-20, 200])
    plt.colorbar()
    plt.xlabel(x.name)
    plt.ylabel(y.name)

plot_scatter(ds.RH, ds.PRECT)

From this plot we can see this is almost the case.

In [ ]:
plot_scatter(ds.PW, ds.PRECT)

The relationship is also visible when PW is used to stratify the precipitation.